In [29]:
from gurobipy import Model, GRB

# Initialize the model
m = Model("Sunnyshore_Financing")

# Define decision variables for loans taken out in each eligible month
loan_terms = ['1', '2', '3']
loans = {(month, term): m.addVar(vtype=GRB.CONTINUOUS, name=f"loan_{month}_{term}m")
         for month in ['May', 'June', 'July']
         for term in loan_terms if not (month == 'July' and term in ['2', '3'])}

# Interest rates for different loan terms
interest_rates = {'1': 0.0175, '2': 0.0225, '3': 0.0275}

# Objective: Minimize the total amount to be repaid
m.setObjective(sum(loans[month, term] * (1 + interest_rates[term]) for month, term in loans), GRB.MINIMIZE)

# Cash flows
initial_cash = 140000
monthly_revenues = {'May': 180000, 'June': 260000, 'July': 420000, 'August': 580000}
monthly_expenses = {'May': 300000, 'June': 400000, 'July': 350000, 'August': 200000}
net_monthly_cash_flows = {month: monthly_revenues[month] - monthly_expenses[month] for month in monthly_revenues}

# Constraints for maintaining minimum cash balance each month
cash_balances = {}
for month in ['May', 'June', 'July', 'August']:
    if month == 'May':
        cash_balance = initial_cash + net_monthly_cash_flows[month]
    else:
        prev_month = list(net_monthly_cash_flows.keys())[list(net_monthly_cash_flows.keys()).index(month)-1]
        cash_balance = cash_balances[prev_month] + net_monthly_cash_flows[month]

    # Add borrowed money and subtract repayments with interest
    for term in loan_terms:
        if (month, term) in loans:
            cash_balance += loans[(month, term)]
        if term != '1' and f"{prev_month}_{term}" in loans:
            cash_balance -= loans[(prev_month, term)] * (1 + interest_rates[term])

    cash_balances[month] = cash_balance
    min_balance_req = {'May': 25000, 'June': 20000, 'July': 35000, 'August': 18000}
    m.addConstr(cash_balance >= min_balance_req[month], name=f"min_cash_{month}")

# Borrowing limits
borrowing_limits = {'May': 250000, 'June': 150000, 'July': 350000}
for month, limit in borrowing_limits.items():
    m.addConstr(sum(loans[(month, term)] for term in loan_terms if (month, term) in loans) <= limit, name=f"borrowing_limit_{month}")

# July cash balance ratio constraint
july_cash_balance = cash_balances['July'] - net_monthly_cash_flows['July']  # Cash balance before July's cash flow
may_june_combined_balance = initial_cash + net_monthly_cash_flows['May'] + net_monthly_cash_flows['June']
m.addConstr(july_cash_balance >= 0.65 * may_june_combined_balance, name="july_cash_balance_ratio")

# Solve the model
m.optimize()

# Display the solution
if m.status == GRB.OPTIMAL:
    print(f"(d) Total amount to repay: ${m.objVal:.2f}")
    print("(e) Money withdrawn in May from all loans:", sum(loans[('May', term)].X for term in loan_terms if ('May', term) in loans))
    print("(f) Cash balance at the end of August:", cash_balances['August'].getValue())
else:
    print("The model is infeasible. Please check the constraints.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 8 rows, 7 columns and 37 nonzeros
Model fingerprint: 0x647dfd9c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+03, 4e+05]
Presolve removed 8 rows and 7 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4245000e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.424500000e+05
(d) Total amount to repay: $142450.00
(e) Money withdrawn in May from all loans: 140000.0
(f) Cash balance at the end of August: 470000.0


In [17]:
# Adjust the June minimum cash balance requirement
min_balances['June'] = 27500  # Increase the minimum cash balance for June to $27,500

# Update the June cash balance constraint in the model
m.remove(m.getConstrByName("min_cash_June"))  # Remove the old constraint
m.addConstr(cash_balances['June'] >= min_balances['June'], name="min_cash_June")  # Add the updated constraint

# Re-optimize the model with the updated constraint
m.optimize()

# Display the new total repayment amount
if m.status == GRB.OPTIMAL:
    print(f"After increasing the minimum cash balance for June to $27,500, the total amount to repay: ${m.objVal:.2f}")
else:
    print("The model is infeasible after the adjustment. Please check the constraints.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 8 rows, 7 columns and 37 nonzeros
Model fingerprint: 0x868751dd
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+03, 4e+05]
Presolve removed 8 rows and 7 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5008125e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.500812500e+05
After increasing the minimum cash balance for June to $27,500, the total amount to repay: $150081.25


In [46]:
from gurobipy import Model, GRB

# Initialize the dual model
dual_model = Model("Dual_Sunnyshore_Financing")

# Define dual variables for each primal constraint
# y variables for cash balance constraints, z variables for borrowing limits, and w for the July cash balance ratio constraint
y = dual_model.addVars(["May", "June", "July", "August"], name="y")
z = dual_model.addVars(["May", "June", "July"], name="z")
w = dual_model.addVar(name="w")

# Define the dual objective function
# The RHS values of primal constraints need to be specified based on the primal LP setup
dual_model.setObjective(
    y["May"] * 25000 + y["June"] * 20000 + y["July"] * 35000 + y["August"] * 18000 +
    z["May"] * 250000 + z["June"] * 150000 + z["July"] * 350000 +
    w * (0.65 * ((140000 + 180000 - 300000) + (140000 + 180000 - 300000 + 260000 - 400000))),
    GRB.MAXIMIZE
)

# Dual constraints corresponding to each primal decision variable
# The contributions of each loan to the primal constraints determine the coefficients in these dual constraints
interest_rates = {'1': 0.0175, '2': 0.0225, '3': 0.0275}
for month in ["May", "June", "July"]:
    for term in ['1', '2', '3']:
        if not (month == 'July' and term in ['2', '3']):  # Exclude invalid loan terms for July
            # Coefficients for dual constraints based on the primal LP structure
            dual_model.addConstr(
                (y[month] if month in y else 0) -
                (y["June"] if "June" in y and term == '1' and month == "May" else 0) -
                (y["July"] if "July" in y and term in ['1', '2'] and month in ["May", "June"] else 0) -
                (y["August"] if "August" in y and term == '1' and month in ["June", "July"] else 0) +
                z[month] +
                (w if month in ["May", "June"] and term == '3' else 0) <=
                1 + interest_rates[term],
                name=f"constraint_{month}_{term}"
            )

# Solve the dual model
dual_model.optimize()

# Output the results
if dual_model.status == GRB.OPTIMAL:
    print(f"Objective Value (Total Value of Resources): {dual_model.ObjVal}")
    for v in dual_model.getVars():
        print(f"{v.VarName}: {v.X}")
else:
    print("Problem could not be solved to optimality.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 7 rows, 8 columns and 23 nonzeros
Model fingerprint: 0xc25075a4
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+04, 4e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.02s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Infeasible or unbounded model
Problem could not be solved to optimality.
